# Group 006-26 Project Report
>Linda Han, Shaqed Orr, Eric Zhang, Prabhjot Singh

## Introduction:
The Iris flower got its name from the goddess of rainbows in Acient Greek mythology, and similar to a rainbow it can be found in diverse colours, shapes and sizes. Despite having over 250 species, we are only interested in 3 specific classes of these perennial plants in our project: Versicolor, Setosa, and Virginica.

In this report, we try to answer the question: given an Iris flower belonging to one of these three classes, how do we predict which class they are in? 

We will be using the `Iris` data set found on https://archive.ics.uci.edu/ml/datasets/iris to build a classifier.

This data set contains 150 samples of data with 3 different classes. Each class contains 50 samples, and four features: sepal length, sepal width, petal length, and petal width. Ultimately, we are trying to predict the class of Iris flowers by these four distinct features using the K-Nearest Neighbor classification method. 

<img src="https://www.almanac.com/sites/default/files/styles/landscape/public/image_nodes/iris-flowers.jpg?itok=yVwKlyWK" width="400" height="200">

*Source: https://www.almanac.com/plant/irises*

## Methods and Results:

### Preliminary data analysis:
---

First, we load in all the necessary libraries.

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(ggplot2)

options(repr.matrix.max.rows = 6) # this lists only 6 rows when we try to display the dataset

#### Reading and cleaning the data

1) We read the `iris` dataset from the original source on the web (UCI Machine Learning Repository) using `read_csv` function

2) We added column names to reflect each of the attributes and mutated the `class` column such that it becomes a factor using `as_factor()`. 

We can now say that our dataset is **tidy** as there is a single iris observation on every row, each column is a single variable (either a measurement of the iris flower or the class it belongs to), and each cell holds a single value.


In [ ]:
iris_col <- c("sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm", "class")
iris <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", col_names= iris_col) %>% 
        mutate(class = as_factor(class))

iris

<br>

#### Splitting the Data

In order to ensure the reproducibility of our results, we use the `set.seed()` function with a randomly chosen initial value. 

There are 150 observations in total and we see from `table 0.1` below that there are 50 observations from each iris class. 

In [ ]:
# Summarizes the number of observations in each class 
# (Iris-setosa, Iris-versicolor, or Iris-virginica)
count <- iris %>%
        count(class)

'Table 0.1'
count

Our goal is to have enough training data points for building an accurate classifier and enough testing data points for making an accurate assessment of the model's performance. Since the proportion of each class is equally represented and the data size isn't large, we choose to partition the data into a 80% training and 20% testing set. This gives us 123 data points for training and 27 data points for testing, which seems like a reasonable partition.

In [ ]:
set.seed(777)

iris_split <- initial_split(iris, prop = 0.80, strata = class)
iris_train <- training(iris_split)
iris_test <- testing(iris_split)

<br>

#### Summary of the training data

Using only the training data, we summarize the data into 2 tables and count the number of rows with missing values to check that all of our rows have values.

1. A summary of the average value of each column using `summarize()` and `across()` 

In [ ]:
iris_avg_size <- iris_train %>%
        summarize(across(sepal_length_cm:petal_width_cm, mean))

'Table 1.1'
iris_avg_size

<br>

2. A summary of the number of observations in each class (Iris-setosa, Iris-versicolor, or Iris-virginica). We observe from this summary that each class has an equal number of data points.

In [ ]:
iris_class_count <- iris_train %>%
        count(class)

'Table 1.2'
iris_class_count

<br>

3. The count for the number of missing rows shows us that all rows in the training dataset have values and will all make meaningful contribution to the building of the classifier.

In [ ]:
sum(is.na(iris_train))

<br>

#### Visualization of the training data

Using only training data, we conduct an exploratory data analysis by visualizing the data with two scatterplots.

For the first graph, we use `ggplot()` to plot the predictors `sepal_width_cm` on the y-axis against `sepal_length_cm` on the x-axis. We also set `colour = "Class"` to mark each observation according to their class. We chose this pair of predictors as their name suggests that they are both measurements of the flower sepal and could therefore be correlated.

To improve readability, we also use the `options()` and `theme()` functions to modify the size of the plot and labels.

In [ ]:
# Graph 1
options(repr.plot.width = 8, repr.plot.height = 6)

iris_plot_sepal <- ggplot(data = iris_train, 
                          aes(x = sepal_length_cm, y = sepal_width_cm , colour = class )) +
                geom_point() +
                labs(x = "Sepal length (cm)", y = "Sepal width (cm)",
                     colour = "Class", caption = "Figure 1.1") +
                ggtitle("Sepal Width vs Sepal Length") +
                theme(text = element_text(size = 20))

iris_plot_sepal

From the graph, we can see the red data points that represent the `Iris-setosa` class form a distinct cluster. Compared to data points from the `Iris-versicolor` and `Iris-virginica` class, the graph suggests that Iris Setosas generally have shorter sepal lengths and longer sepal widths. Meanwhile, the data points for `Iris-versicolor` and `Iris-virginica` seem to be blended together.

<br>

For the second graph, we repeat the same things we did for graph 1 except for changing the two predictors to `petal_length_cm` and `petal_width_cm`.

In [ ]:
# Graph 2
iris_plot_petal <- ggplot(data = iris_train,
                          aes(x = petal_length_cm, y = petal_width_cm , colour = class )) +
                    geom_point() +
                    labs(x = "Petal length (cm)", y = "Petal width (cm)",
                         colour = "Class", caption = "Figure 1.2") +
                    ggtitle("Petal Width vs Petal Length") +
                    theme(text = element_text(size = 20))

iris_plot_petal

Based on the colors, we observe 3 regions and their distinctiveness suggest that we can classify irises based on their petal length and petal width.

Petal width (in ascending order): `Iris-setosa` < `Iris-versicolor` < `Iris-virginica`

Petal length (in ascending order): `Iris-setosa` < `Iris-versicolor` < `Iris-virginica`

From the ranking above, we can see that `Iris-setosa` has the shortest petal width and length, `Iris-virginica` has the longest petal width and length, and `Iris-versicolor` tends to be somewhere in between.

### Model Training Phase (Building The K-Nearest Neighbour Classifier)

----------------------------

#### 1. Recipe 

We choose to use all non-categorical variables as predictors since all of them depict crucial measurements of the iris flower.

Since the K-nearest neighbors algorithm is sensitive to the scale of the predictors, we use `step_center()` and `step_scale()` with the `all_predictors()` argument passed in to standardize our data.

In [ ]:
iris_recipe <- recipe(class ~ ., data = iris_train) %>% 
            step_center(all_predictors()) %>% 
            step_scale(all_predictors()) 

iris_recipe

<br>

To visualize the scaled `iris_train` data, we use the `prep()` and `bake()` functions

In [ ]:
iris_scaled <- iris_recipe %>% 
                prep() %>% 
                bake(iris_train)

'Figure 1.3'
iris_scaled

<br>

#### 2. Model Specification

Now we create a nearest neighbour model for our training set using `nearest_neighbor()` and setting the neighbors argument to `tune()` so we can proceed to find the optimal $K$ value. 

As this is a KNN classification problem, we set the engine to `"kknn"` and the mode to `"classification"`

In [ ]:
iris_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>% 
            set_engine("kknn") %>% 
            set_mode("classification")

iris_spec

<br>

#### 3. Vfold

When it comes to choosing the number of folds for performing cross-validation, we need to take into account that having too few folds (such as 1,2) could result in a biased estimation of our model performance. On the other hand, having too many folds (such as 20) could increase the runtime and cause the overall process to be computationally expensive. Taking that into consideration, choosing $K = 5$ folds seems to be reasonable.



In [ ]:
iris_vfold <- vfold_cv(iris_train, v = 5, strata = class)

<br>

#### 4. Workflow

We create a workflow and use the `tune_grid()` function to try $K$ values from 1 to 20 by passing in a tibble object that we created called `k_vals`. 

In [ ]:
k_vals <- tibble(neighbors = seq(from=1, to=20, by=1))

iris_tune_workflow <- workflow() %>% 
    add_recipe(iris_recipe) %>% 
    add_model(iris_spec) %>% 
    tune_grid(resamples = iris_vfold, grid = k_vals)

<br>

#### 5. Collect Metrics

From `iris_tune_workflow`, we use the `collect_metrics()` function and filter for `accuracy` from the `.metric` column. However, this gives us a table of unordered rows. To select the `neighbors` value that gives us the highest accuracy, we use `arrange()` with `desc()` to arrange the rows by their `mean` column in descending order.

In [ ]:
iris_metrics_tune <- iris_tune_workflow %>% 
    collect_metrics() %>% 
    filter(.metric == "accuracy") %>% 
    arrange(desc(mean))

'Figure 5.1'
iris_metrics_tune

We see from the first three rows of Figure 5.1 above that $K = 7,8,9$ all yield the highest accuracy of 96%.

<br>

#### 6. Plot Neighbors vs Accuracy Graph

To visualize Figure 5.1, we create a scatterplot with `neighbors` on the x-axis and `mean` on the y-axis. The `geom_line()` function connects all the points with lines and the `scale_x_continuous()` is used to to adjust the x-axis.

In [ ]:
neighbors_plot <- ggplot(iris_metrics_tune, aes(x=neighbors, y=mean)) + 
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate", caption="Figure 6.1") + 
    ggtitle("Accuracy vs K") +
    theme(text = element_text(size = 20)) +
    scale_x_continuous(breaks = seq(0, 25, by = 2))

neighbors_plot

We want our model to have the right amount of sensitivity and avoid under or overfitting. From Figure 6.1 above, we see that the accuracy estimate peaks from $K=7$ neighbors to $K=14$ neighbors. Therefore, **We choose the optimal $K$ to be $7$** as it is the point where the accuracy estimate first peaked.


<br>

#### 7. Rebuild Model Specification and Workflow with $K$ = 7

With the optimal $K$ found, we rebuild the model with the `neighbors` argument equal to $7$ and name it `iris_spec_optimal`. We also create a new workflow object called `iris_fit` that uses the new model.

In [ ]:
iris_spec_optimal <- nearest_neighbor(weight_func = "rectangular", neighbors = 7) %>% 
            set_engine("kknn") %>% 
            set_mode("classification")

iris_spec_optimal

iris_fit <- workflow() %>% 
    add_recipe(iris_recipe) %>% 
    add_model(iris_spec_optimal) %>% 
    fit(iris_train)

iris_fit

<br> 

### Model Testing Phase

------------------------

#### 1. Predict

We can now use the classifier to predict labels for our test dataset by using the `predict()` function and `bind_cols()` to bind the predicted class onto the test dataset for comparison.

In [ ]:
iris_test_predictions <- predict(iris_fit, iris_test) %>% 
    bind_cols(iris_test)

iris_test_predictions

#### 2. Assess accuracy of predictions

First, we use `metrics` to check the accuracy of our predictions for the iris labels in the test set. We see that we have achieved 96% accuracy, which is the same as the accuracy estimate for our optimal $K=7$ model during training phase.

In [ ]:
iris_metrics_final <- iris_test_predictions %>% 
    metrics(truth = class, estimate = .pred_class)

iris_metrics_final

<br>

We can also summarize the prediction outcomes by using a confusion matrix. By looking at the values in the matrix, we are given more detail about the reason for our 96% accuracy as we see that one `Iris-virginica` is mislabeled as an `Iris-versicolor`.

In [ ]:
iris_conf_mat <- iris_test_predictions %>% 
    conf_mat(truth = class, estimate = .pred_class)

iris_conf_mat

<br>

#### 3. Visualization of Data Analysis Result

Created a mosaic plot ... TODO ... add more comments

In [ ]:
mos_data<- iris_test_predictions %>%
    semi_join(iris_test) %>%
    group_by(class, .pred_class)%>%
    summarize(n=n())

mos_data

In [ ]:
mos_plot <- ggplot(mos_data, aes(x=class, y=n, fill=.pred_class))+
        geom_bar(position="fill", stat="identity")+
        labs(x="Classes of Iris Flower", y="Count",
             fill="Iris Classes", caption="Figure 2.3") +
        ggtitle("Predictions of Iris Flowers") +
        theme(text = element_text(size = 15))

mos_plot

<br>

## Discussion:

#### A summary of our findings:


<br>

## References: